# VGG

在这里我们使用TensorFlow来实现VGG-16,但是由于VGG-16模型还是比较大的,奈何炼丹炉装不下,所以我们会使用官方已经训练完毕(基于ImageNet)的参数做一个简单的迁移学习(微调).

模型参数来自于:

预训练权值由[VGG at Oxford](http://www.robots.ox.ac.uk/~vgg/research/very_deep/)发布的预训练权值移植而来,基于[Creative Commons Attribution License](https://creativecommons.org/licenses/by/4.0/).

在开始之前,我们回顾一下VGG结构:

![](../../../../picture/260.png)

### 1. LoadData

当然,第一件事情还是处理数据,这里我们依然使用Stanford的[stanford-dogs-dataset](https://www.kaggle.com/jessicali9530/stanford-dogs-dataset),该数据集含有20,580张不同种类狗的图片,共120类.在Annotation文件中包含了对应图片的类别以及bounding box的参数,但是这个数据集不均匀,由于时间限制我没有做过多的数据预处理,也就是说模型最后会过拟合.各个类别详情查看[ImageNetDogs](http://vision.stanford.edu/aditya86/ImageNetDogs/).



对于数据处理,我们这里有两个文件,下面分别对这两个文件做一个简单的说明:

**A.[Load_Flies](Load_Flies.py):**

该文件主要用来加载Images和Labels,其中Labels有两个选择(a).labels(分类);(b).bboxs(边框),通过参数```target_mode```指定.另外需要注意的是,无论选择的是何种Labels,最终只返回Images(图片路径,array)和labels(array).

下面来看看函数详情:

A1:```load_files:```加载数据的启动函数,选择加载分类还是边框,如果选择分类(labels)则运行```__load_files_labels```,否则运行```__load_files_bbox```,当运行结束之后将数据打乱并保存```samples_test```数据以便于评估时候使用.

A2:```__load_files_bbox:``` 用于加载标签为bbox,需要注意的是,这里采用的是(per-classregression,PCR)模式,也就是说我们只加载一类,没办法炼丹炉以及时间不允许,另外在加载bbox的时候我做了一点小操作,将原来的```xmin,ymin,xmax,ymax```进行百分比处理,这样处理的好处是对于不同尺度的预测图,我们无需再进行一次映射转换(也就是OverFeat中的操作).还有一点是将百分比处理后的4个参数进行了log处理,来保证后面回归网络的加速(后面会说明)

**B.[LoadDataset](LoadDataset.py):**

该文件和往常一样使用TensorFlow的Dataset来加载数据,但是需要注意的是,该文件一样可以直接处理分类或者边框,返回的是一个```next_element```用来迭代生成数据

下面来看看函数详情:

B1:```get_batch:```用来判断加载的是分类还是边框,因为我们知道分类是需要将标签进行One-hot处理的而边框则不需要,无论选择执行哪一个,都需要保证验证集是干净的,也就是说验证集是不进行数据增广的,另外需要注意的是,我没有将数据做Normal(实际上我不喜欢将数据做Normal),只是简单的除以255以保证梯度更新.

### 2. Classification 

现在我们来看看如何使用预训练好的VGG-16做微调.

**A.[CVGG16](CVGG16.py):**

文件中的注释已经写的非常清楚,需要注意的是,我使用的**尺度抖动[256,512]**来训练,使用随机裁剪$(224 \times 224)$,但是没有做RGB处理,另外这里我们来说明一下几个重要的函数

A1:```Keras_Vgg16:```我这里是使用Keras中预训练好的模型,详细地址[Keras-VGG](https://keras.io/zh/applications/#vgg16),需要注意的是在官方示例中使用了这段代码```x = preprocess_input(x)```,该函数完成数据预处理的工作,数据预处理能够提高算法的运行效果,常用的预处理包括数据归一化和白化(whitening),然而我们在LoadData中已经做过了,所以这里绝对不能使用,否则你会发现在测试的时候网络不work.

A2:```Vgg:```由于我们的数据是先经过```Keras_Vgg16```到最后一个池化层,所以我们在初始化```data```的```placeholder```的时候```shape```为$(None, 7, 7, 512)$.

当模型训练完毕之后,将文件保存在本地即可,这里我们附上模型训练的过程,我设计训练正确率(top-5)只要达到0.8和验证集正确率只要达到0.6即保存模型:

![](../filps/CVGGtop_5.png)

**B.[CTesting](CTesting.py):**

当训练完毕之后,我们就可以按照paper的方法进行评估,需要注意的是,我这里没有采用多裁剪,仅仅使用的是多尺度$(224,256)$.需要注意的是,对于分类评估,我是将所有每一张图片的所有尺度下获取top-5,然后将所有尺度下的结果保存,最后再依靠频数(也就是每个尺度下出现最多的标签)获取top-5的结果.

下面来看看重要函数:

B1.```Testing:```包含两个选项:(a).```Evaluate```用于评估测试集(由于算法效率太低,我没有运行,实际上运行之后发现计算机资源会很快消磨殆尽);(b).```Predict```用于预测单张图片

B2.```Reshape_weights:```由于我们在测试阶段使用的是全卷积网络,所以我们需要将FC的所有weights,bias更改形状以保证全卷积的运行.

B3.```ScalePredict:``` 获取当前尺度下的预测值.

B4.```STop_k:```用于获取当前尺度下的top-5标签,需要注意的是我们经过全卷积后的形状是$(N\times S\times S \times C)$的形状,所以我们还需要将形状转换为$(N \times C)$

B5.```Top_k:```当所有尺度下的```STop_k```执行完毕,会保存一个巨大的$(N \times C)$矩阵,此时我们依照频数获取最终top-5的预测值

下面让我们来看看结果:

<table border="0">
	<tbody>
    <tr>
			<td style="text-align: center"><strong>toy_terrier</strong></td>
			<td style="text-align: center"><strong>African_hunting_dog</strong></td>
            <td style="text-align: center"><strong>Pembroke</strong></td>
            <td style="text-align: center"><strong>Pug</strong></td>
	</tr>
    <tr>
			<td > <img src="../filps/CT1.png" style="max-width:100%;"></td>
			<td > <img src="../filps/CT2.png" style="max-width:100%;"></td>
            <td > <img src="../filps/CT3.png" style="max-width:100%;"></td>
            <td > <img src="../filps/CT4.png" style="max-width:100%;"></td>
	</tr>
	</tbody>
</table>

### 3. Regression

接下来我们来看看如何使用预训练的VGG-16来微调回归网络

**A.[RVGG16](RVGG16.py):**

回归网络和分类基本相似,由于我们这里使用的是PCR模式,所以我们一次只能训练一个类别,由于在数据集中African_hunting_dog的类别最多,所以我选择训练这个类,需要注意的是,这里损失函数我选择使用```huber_loss```,该损失函数比MSE更加优秀,更多损失函数请查看[常用回归损失函数](https://www.jiqizhixin.com/articles/2018-06-21-3).

现在我们来说说为什么我们在加载bbox的时候要处理一次log,由于我们做的是回归操作,那么在最开始的时候,最常见的预测bbox会有正有负,因为我们的bbox只可能是整数,此时如果我们使用exp(函数单调)就可以将预测bbox全部转换为整数,同时也可以想象如果真实的坐标是1,那么预测出来即使是-1也应该是可以的,因为对于图像坐标而言正负号是没有意义的,所以按照这个逻辑往回推,我们的初始bbox就需要取log,来达到加速拟合的效果.用比较官方的话来说,当预测值(Predict truth: PT)和真实值(Ground Truth:GT)比较相近的时候,log(PT,GT)将会是一个线性函数.


A1.```Iou:```这里提供的是Numpy版本,在接下去的模型中会提供Tensor版本

训练结果如下(当验证集训练平均IOU>0.6):


![](../filps/RVGG.png)


**B.[RTesing](RTesing.py):**

最后让我们来看看回归网络的效果,需要注意的是,这里我没有使用多尺度/多裁剪预测,贪心算法实在恶心,也就是我只是用单尺度评估,效果实际上还行.

B1.```PlotImage:```需要注意的是,在预测出来的4-D向量为```[xmin, ymin, xmax, ymax]```的形式而不是paper中```[conter_x, conter_y, width, height]```的形式,且其实百分比制所以我们需要乘上图片的宽高来获取真正的边框

下面我们来看看效果:

<table border="0">
	<tbody>
    <tr>
			<td style="text-align: center"><strong>Good</strong></td>
			<td style="text-align: center"><strong>Common</strong></td>
            <td style="text-align: center"><strong>Poor</strong></td>
            <td style="text-align: center"><strong>Poor</strong></td>
	</tr>
    <tr>
			<td > <img src="../filps/RT1.png" style="max-width:100%;"></td>
			<td > <img src="../filps/RT2.png" style="max-width:100%;"></td>
            <td > <img src="../filps/RT3.png" style="max-width:100%;"></td>
            <td > <img src="../filps/RT4.png" style="max-width:100%;"></td>
	</tr>
	</tbody>
</table>

至此,VGG-16也就结束了,之后的模型我们会尝试进行多目标检测.